In [8]:
import numpy as np
import pandas as pd
import gurobipy as gp
from gurobipy import GRB

from pathlib import Path
import os
import sys
from datetime import datetime, date


In [9]:
result = Path('result/')
report = Path('report/')
instances = Path('instances/csifa')

In [11]:
def read_instance(file_name):
	
	arq = open(file_name)
	
	N = int(arq.readline())
	
	PR  = [0]*N
	PP  = [0]*N

	FR = [float(arq.readline())]*N
	FP = [float(arq.readline())]*N

	HR = [float(arq.readline())]*N
	HP = [float(arq.readline())]*N

	D = [int(i) for i in arq.readline().split()]
	
	R = [int(i) for i in arq.readline().split()]

	C = float(arq.readline().rstrip('\n'))
	
	return N, PP, PR, FP, FR, HR, HP, D, R, C

In [12]:
MAX_CPU_TIME = 3600.0
EPSILON = 1e-6

def clsr_std_mip(N, PP, PR, FP, FR, HR, HP, D, R, SD, SR, C):
	try:

		# Create a new model
		model = gp.Model("clspr_std")

		# Create variables
		xp = model.addVars(list(range(N)), vtype=GRB.CONTINUOUS, name="xp")
		yp = model.addVars(list(range(N)), vtype=GRB.BINARY, name="yp")
		sp = model.addVars(list(range(N)), vtype=GRB.CONTINUOUS, name="sp")
		xr = model.addVars(list(range(N)), vtype=GRB.CONTINUOUS, name="xr")
		yr = model.addVars(list(range(N)), vtype=GRB.BINARY, name="yr")
		sr = model.addVars(list(range(N)), vtype=GRB.CONTINUOUS, name="sr")
		
		model.update()

		# set objective

		#model.setObjective(gp.quicksum(
		#	PP[i]*xp[i] + HP[i]*sp[i] + FP[i]*yp[i] +  
		#	xr[i]*PR[i] + sr[i]*HR[i] + yr[i]*FR[i] for i in range(N)), sense = GRB.MINIMIZE)

		fobj = gp.quicksum(PP[i]*xp[i] for i in range(N))
		fobj += gp.quicksum(HP[i]*sp[i] for i in range(N))
		fobj += gp.quicksum(FP[i]*yp[i] for i in range(N))
		fobj += gp.quicksum(PR[i]*xr[i] for i in range(N))
		fobj += gp.quicksum(HR[i]*sr[i] for i in range(N))
		fobj += gp.quicksum(FR[i]*yr[i] for i in range(N))
	
		model.setObjective(fobj, sense = GRB.MINIMIZE)

		# add constraints

		model.addConstr(xp[0] + xr[0] - sp[0] == D[0])

		model.addConstrs(sp[i-1] + xp[i] + xr[i] - sp[i] == D[i] for i in range(N) if i > 0 )
		
		model.addConstr(R[0] - xr[0] - sr[0] == 0)
		
		model.addConstrs(sr[i-1] + R[i] - xr[i] - sr[i] == 0 for i in range(N) if i > 0)
		
		model.addConstrs(xp[i] - yp[i]*min(SD[i][N-1],C) <= 0 for i in range(N))
		
		model.addConstrs(xr[i] - yr[i]*min(SR[0][i],SD[i][N-1],C) <= 0 for i in range(N))
		
		model.addConstrs(xp[i] + xr[i] <= C for i in range(N))
	    
		# export .lp
		#model.write(file_name+"_model.lp")

		# parameters 
		model.setParam(GRB.Param.TimeLimit, MAX_CPU_TIME)
		model.setParam(GRB.Param.MIPGap, EPSILON)
		model.setParam(GRB.Param.Threads, 1)
		#model.setParam(GRB.Param.Cuts, -1)
		#model.setParam(GRB.Param.Presolve, -1)

		# turn off display
		gp.setParam('OutputFlag', 0)

		# open log file
		_ = open('clspr_std.log', 'w')

		# optimize model
		model.optimize()
		
		tmp = 0
		if model.status == GRB.OPTIMAL:
			tmp = 1


	except gp.GurobiError as e:
		print('Error code ' + str(e.errno) + ': ' + str(e))
	
	objval = model.ObjVal
	objbound = model.ObjBound
	mipgap = model.MIPGap
	runtime = model.Runtime
	nodecount = model.NodeCount

	return objval, objbound, mipgap, runtime, nodecount, tmp

In [13]:
def main(file_name):
	
	N, PP, PR, FP, FR, HR, HP, D, R, C = read_instance(os.path.join(instances,file_name))

	SD = (np.zeros((N,N))).tolist()
	SR = (np.zeros((N,N))).tolist()

	for  i in range(N):
		SD[i][i] = D[i]
		SR[i][i] = R[i]
		for j in range(i+1,N):
			SD[i][j] = SD[i][j-1] + D[j]
			SR[i][j] = SR[i][j-1] + R[j]

	bestsol, bestbound, gap, runtime, numnode, tmp = clsr_std_mip(N, PP, PR, FP, FR, HR, HP, D, R, SD, SR, C)
	
	arquivo = open(os.path.join(result,'clsr_std_mip.csv'),'a')
	
	arquivo.write(file_name+';'
			   +str(round(bestsol,3))+';'
			   +str(round(bestbound,3))+';'
			   +str(round(gap,3))+';'
			   +str(round(runtime,3))+';'
			   +str(round(numnode,3))+';'
			   +str(round(tmp,3))+'\n')

	arquivo.close()

In [14]:
if __name__== "__main__" :

	for dim in [52]:
		for id in range(12,25):
			file_name = f"c{dim}_{id}.txt"			
			main(file_name)